In [1]:
import os
import zarr
import xarray as xr
import subprocess as subp
from rechunker import rechunk
from dask.diagnostics import ProgressBar
from dask.distributed import Client, LocalCluster

In [2]:
cluster = LocalCluster(
    dashboard_address=":8788", threads_per_worker=1, n_workers=10, memory_limit="30GiB"
)
client = Client(cluster)

In [3]:
client

Client Scheduler: tcp://127.0.0.1:41836 Dashboard: http://127.0.0.1:8788/status,Cluster Workers: 10 Cores: 10 Memory: 300.00 GiB


In [4]:
!rm -r /work/scratch-pw/mattjbr/chess_scape/yearchunk/*

rm: cannot remove ‘/work/scratch-pw/mattjbr/chess_scape/yearchunk/*’: No such file or directory


In [ ]:
ensmems = ["01", "04", "06", "15"]
varnames = [
    "hurs",
    "huss",
    "pr",
    "psurf",
    "rlds",
    "rsds",
    "sfcWind",
    "tmax",
    "tmean",
    "tmin",
]

maxmem = "20GB"
tempstore = "/work/scratch-pw/mattjbr/chess_scape/tempstore.zarr"
if not os.path.exists(os.path.dirname(tempstore)):
    os.makedirs(os.path.dirname(tempstore))

for ensmem in ensmems:
    for varname in varnames:
        if ensmem != "01":
            if varname == "psurf":
                continue

        print("Processing " + ensmem + " " + varname)

        source_group = zarr.open(
            "/gws/nopw/j04/ceh_generic/matbro/chess_scape/year100kmchunk/ens"
            + ensmem
            + "-year100kmchunk/"
            + varname
            + "_"
            + ensmem
            + "_year100km.zarr"
        )
        targetstore = (
            "/work/scratch-pw/mattjbr/chess_scape/100year10kmchunk/"
            + varname
            + "_"
            + ensmem
            + "_100year10km.zarr"
        )

        if not os.path.exists(os.path.dirname(targetstore)):
            os.makedirs(os.path.dirname(targetstore))

        if varname == "tmean":
            zarrvname = "tas"
        elif varname == "tmax":
            zarrvname = "tasmax"
        elif varname == "tmin":
            zarrvname = "tasmin"
        else:
            zarrvname = varname

        target_chunks = {
            zarrvname: {"time": 36000, "y": 10, "x": 10},
            "lat": {"y": 10, "x": 10},
            "lon": {"y": 10, "x": 10},
            "y": {"y": 10},
            "x": {"x": 10},
            "time": {"time": 36000},
        }

        subp.call(["rm " + tempstore], shell=True)
        array_plan = rechunk(
            source_group, target_chunks, maxmem, targetstore, temp_store=tempstore
        )
        array_plan.execute()

Processing 01 hurs


distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker


Processing 01 huss


distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker


Processing 01 pr


distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker


Processing 01 psurf
Processing 01 rlds
Processing 01 rsds
Processing 01 sfcWind
Processing 01 tmax
Processing 01 tmean
Processing 01 tmin
Processing 04 hurs
Processing 04 huss
Processing 04 pr
Processing 04 rlds
Processing 04 rsds
Processing 04 sfcWind
Processing 04 tmax
Processing 04 tmean
Processing 04 tmin
Processing 06 hurs


---------------------

Optimum chunk size for S3 store is about 250MB, work out what this corresponds to in terms of chunk sizes if just chunking in time

In [22]:
import math

In [61]:
data = xr.open_zarr(
    "/gws/nopw/j04/ceh_generic/matbro/chess_scape/monthchunk/ens01-monthchunk/tmean.zarr"
)

In [62]:
data

<xarray.Dataset>
Dimensions:  (time: 36000, x: 656, y: 1057)
Coordinates:
  * time     (time) object 1980-12-01 12:00:00 ... 2080-11-30 12:00:00
  * x        (x) float32 500.0 1.5e+03 2.5e+03 ... 6.535e+05 6.545e+05 6.555e+05
  * y        (y) float32 500.0 1.5e+03 2.5e+03 ... 1.054e+06 1.056e+06 1.056e+06
Data variables:
    lat      (y, x) float32 dask.array<chunksize=(1057, 656), meta=np.ndarray>
    lon      (y, x) float32 dask.array<chunksize=(1057, 656), meta=np.ndarray>
    tas      (time, y, x) float32 dask.array<chunksize=(30, 1057, 656), meta=np.ndarray>
Attributes:
    NCO:            4.7.3
    contact:        emrobi@ceh.ac.uk
    institution:    CEH Wallingford - NERC
    normalisation:  bias corrected against historical CHESS data
    project:        UK-SCAPE: SPEED

In [31]:
nmbytes = data["tas"].nbytes / 1e6

In [78]:
chunk_size_mbytes = 250
time_chunk = math.ceil(len(data.time) / math.ceil(nmbytes / chunk_size_mbytes))
print(time_chunk)

90


Now calculate chunk sizes for other configurations

In [64]:
def calc_chunk_size(tsize, xsize, ysize):
    return (tsize * xsize * ysize * 4) / 1e6

In [65]:
calc_chunk_size(30, 656, 1057)  # month chunks, no space

83.20704

In [66]:
calc_chunk_size(90, 656, 1057)  # season chunks, no space

249.62112

In [67]:
calc_chunk_size(360, 656, 1057)  # year chunks, no space

998.48448

In [69]:
calc_chunk_size(360, 100, 100)  # year, 100km chunks

14.4

In [70]:
calc_chunk_size(3600, 100, 100)  # decade, 100km chunks

144.0

In [77]:
calc_chunk_size(36000, 10, 10)  # century, 10km chunks

14.4